In [1]:
! pip install implicit

     |████████████████████████████████| 1.1MB 8.1MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419464 sha256=54dc9318c60c2cd83b3fb991a74e708ea2b92b5c016192542926282b0f3922eb
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [2]:
import os
import pandas as pd

# rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
rating_file_path='/content/drive/MyDrive/DL_Study/AIFFEL/recommend/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path='/content/drive/MyDrive/DL_Study/AIFFEL/recommend/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()
# 가장 인기 있는 영화 30개(인기순)
rating_count = ratings.groupby('movie_id')['count'].count()
rating_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: count, dtype: int64

In [4]:
my_favorite= [32, 260, 286, 316, 329]
# my_favorite = ['deep impact' , 'amagedon' ,'cast away' ,'matrix' ,'three rings']
my_user_id = ratings.tail(1)['user_id'].item() + 1
print(my_user_id)
my_favorite_list = pd.DataFrame({'user_id': [my_user_id]*5, 'movie_id':my_favorite,
                                 'count':[5]*5, 'timestamp': [978301777]*5})
ratings = ratings.append(my_favorite_list)
ratings.tail()

6041


,user_id,movie_id,count,timestamp
0,6041,32,5,978301777
1,6041,260,5,978301777
2,6041,286,5,978301777
3,6041,316,5,978301777
4,6041,329,5,978301777


In [5]:
# 4) CSR matrix를 직접 만들어 봅시다.
from implicit.als import AlternatingLeastSquares
import os
import numpy as np
from scipy.sparse import csr_matrix

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

In [6]:
# csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)),
                      # shape= (num_user, num_movie))
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False
                                    , iterations=15, dtype=np.float32)


In [7]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [8]:
als_model.fit(csr_data_transpose)

In [9]:
# jungle book 영화 선택
my_user_vector, movie_recommend = als_model.user_factors[my_user_id], als_model.item_factors[362]
 
np.dot(my_user_vector, movie_recommend)

0.0028388533

In [10]:
# 내가 선택한 영화와 비슷한 영화 리스트 보기
def get_similar_movie(movie_id: int):
  similar_movie = als_model.similar_items(movie_id)
  ml = [i[0] for i in similar_movie]
  return ml

# 480 === 쥐라기공원
movie_recommended = get_similar_movie(480)
movie_recommended


[480, 1580, 589, 2916, 2571, 110, 989, 1544, 648, 268]

In [13]:
def id_to_moviename(movie_id: list):
  title = movies[movies['movie_id'].isin(movie_id)]

  return title['title']

print(id_to_moviename(movie_recommended))

108                             Braveheart (1995)
265                          Little Odessa (1994)
476                          Jurassic Park (1993)
585             Terminator 2: Judgment Day (1991)
642                    Mission: Impossible (1996)
977     Schlafes Bruder (Brother of Sleep) (1995)
1505        Lost World: Jurassic Park, The (1997)
1539                          Men in Black (1997)
2502                           Matrix, The (1999)
2847                          Total Recall (1990)
Name: title, dtype: object


In [ ]:
#오 쥐라기공원을 선택하니 전부다 내가 좋아하는 영화가 나오네~
